In [ ]:
# ✅ STEP 1: Install Gradio
!pip install -q gradio

# ✅ STEP 2: Import Libraries
import os
import cv2
import numpy as np
import gradio as gr
from PIL import Image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Conv2D, MaxPooling2D, BatchNormalization,
                                     Dropout, Dense, GlobalAveragePooling2D)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from google.colab import drive

# ✅ STEP 3: Mount Google Drive
drive.mount('/content/drive')

# ✅ STEP 4: Constants
IMG_SIZE = 128
BATCH_SIZE = 16
EPOCHS = 10
DATASET_PATH = '/content/drive/MyDrive/brain_tumor/Training'
CATEGORIES = ['glioma', 'meningioma', 'notumor', 'pituitary']

# ✅ STEP 5: Load and preprocess data
data = []
labels = []

for category in CATEGORIES:
    folder_path = os.path.join(DATASET_PATH, category)
    print(f"Loading images for: {category}")
    for img_name in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_name)
        try:
            image = cv2.imread(img_path)
            if image is not None:
                image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
                data.append(image)
                labels.append(category)
        except Exception as e:
            print(f"Skipped {img_path}: {e}")

# ✅ Convert to NumPy arrays and normalize
data = np.array(data) / 255.0

# ✅ Encode labels
le = LabelEncoder()
labels_encoded = le.fit_transform(labels)
labels_categorical = to_categorical(labels_encoded, num_classes=len(CATEGORIES))

# ✅ Train-test split
X_train, X_test, y_train, y_test = train_test_split(data, labels_categorical, test_size=0.2, random_state=42)

# ✅ STEP 6: Build CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D((2, 2)),
    BatchNormalization(),

    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    BatchNormalization(),

    Conv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    BatchNormalization(),

    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(CATEGORIES), activation='softmax')
])

# ✅ Compile model
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# ✅ Train model
history = model.fit(X_train, y_train, batch_size=BATCH_SIZE,
                    validation_data=(X_test, y_test), epochs=EPOCHS, verbose=1)

# ✅ STEP 7: Prediction function for Gradio
def predict_image(image):
    image = image.resize((IMG_SIZE, IMG_SIZE))
    image = np.array(image)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    image = image / 255.0
    image = np.expand_dims(image, axis=0)
    prediction = model.predict(image)
    label = le.inverse_transform(np.argmax(prediction, axis=1))[0]
    return {cat: float(prediction[0][i]) for i, cat in enumerate(le.classes_)}

# ✅ STEP 8: Gradio interface
interface = gr.Interface(
    fn=predict_image,
    inputs=gr.Image(type='pil'),
    outputs=gr.Label(num_top_classes=4),
    title="Brain Tumor Classifier",
    description="Upload a brain MRI image to classify it as glioma, meningioma, notumor, or pituitary."
)

# ✅ STEP 9: Launch Gradio app
interface.launch(share=True)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loading images for: glioma
Loading images for: meningioma
Loading images for: notumor
Loading images for: pituitary


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 35s 758ms/step - accuracy: 0.8331 - loss: 0.8239 - val_accuracy: 1.0000 - val_loss: 0.0774
Epoch 2/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 29s 745ms/step - accuracy: 1.0000 - loss: 0.0876 - val_accuracy: 1.0000 - val_loss: 2.2213e-04
Epoch 3/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 38s 657ms/step - accuracy: 1.0000 - loss: 0.0104 - val_accuracy: 1.0000 - val_loss: 6.0216e-06
Epoch 4/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 41s 669ms/step - accuracy: 1.0000 - loss: 0.0068 - val_accuracy: 1.0000 - val_loss: 5.3876e-07
Epoch 5/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 41s 664ms/step - accuracy: 1.0000 - loss: 0.0041 - val_accuracy: 1.0000 - val_loss: 5.6508e-08
Epoch 6/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 44s 749ms/step - accuracy: 1.0000 - loss: 0.0019 - val_accuracy: 1.0000 - val_loss: 3.0963e-09
Epoch 7/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 40s 716ms/step - accuracy: 1.0000 - loss: 0.0019 - val_accuracy: 1.0000 - val_loss: 1.6256e-08
Epoch 8/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 41s 737ms/step - accuracy: 1.0000 -